# Video selection to train DLC networks

We want to track LEDs and the center of the head in the 2 videos we recorded.

* `arena_top`
* `positrack2`


In [1]:
%load_ext autoreload
%autoreload 2
%run setup_project.py

/home/kilo/repo/analysis_jingjie/jingjie/autopi_mec_jingjie/autopi_mec/mouse_and_objects_position_tracking/setup_project.py:769: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()
/home/kilo/repo/analysis_jingjie/jingjie/autopi_mec_jingjie/autopi_mec/mouse_and_objects_position_tracking/setup_project.py:797: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [3]:
projectName, dataPath, dlcModelPath, myProject, sSessions= setup_project_session_lists()

creating myProject, an autopipy.project object
Project name: autopi_mec
dataPath: /adata/projects/autopi_mec
dlcModelPath: /adata/models
Reading /adata/projects/autopi_mec/sessionList_invalid
We have 1 testing sessions in the list
spikeA.Kilosort_session objects are in sSessions


## AutoPI arena_top videos

We just want a subset of recording sessions to use as training material.

We will use 8 videos, which should give us enough variability. 

In [5]:
sessionIndices = [0]
for i in sessionIndices:
    ses = myProject.sessionList[i]
    print(ses.fileNames["arena_top.cropped.avi"])

/adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/mn8578-30112021-0107.arena_top.cropped.avi


## Create shorter videos

It is easier to work with shorter videos, as I will be training off-site.

In [6]:
for i in sessionIndices:
    ses = myProject.sessionList[i]
    inputFile = ses.fileNames["arena_top.cropped.avi"]
    outputFile = "{}/{}{}".format(ses.path,ses.name,".arena_top.cropped_short.avi")
    myCMD = "ffmpeg -i {} -vb 10M -ss 00:10:00 -t 00:20:00 {}".format(inputFile,outputFile)
    print(myCMD)
    os.system(myCMD)

ffmpeg -i /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/mn8578-30112021-0107.arena_top.cropped.avi -vb 10M -ss 00:10:00 -t 00:20:00 /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/mn8578-30112021-0107.arena_top.cropped_short.avi


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

We can copy these shorter files into a single directory to make the copying across computer easier. 

In [13]:
for i in sessionIndices:
    ses = myProject.sessionList[i]
    originFile = "{}/{}{}".format(ses.path,ses.name,".arena_top.cropped_short.avi")
    destinationFolder = "/adata/projects/autopi_mec/results/training_videos_arena_top/"
    
    myCMD = "cp {} {}".format(originFile,destinationFolder)
    print(myCMD)
    os.system(myCMD)

cp /adata/projects/autopi_mec/mn8578/mn8578-30112021-0107/mn8578-30112021-0107.arena_top.cropped_short.avi /adata/projects/autopi_mec/results/training_videos_arena_top/
cp /adata/projects/autopi_mec/mn8578/mn8578-25122021-0107/mn8578-25122021-0107.arena_top.cropped_short.avi /adata/projects/autopi_mec/results/training_videos_arena_top/
cp /adata/projects/autopi_mec/mn8599/mn8599-01022022-0108/mn8599-01022022-0108.arena_top.cropped_short.avi /adata/projects/autopi_mec/results/training_videos_arena_top/
cp /adata/projects/autopi_mec/jp1084/jp1084-14042022-0108/jp1084-14042022-0108.arena_top.cropped_short.avi /adata/projects/autopi_mec/results/training_videos_arena_top/
cp /adata/projects/autopi_mec/jp1685/jp1685-24042022-0108/jp1685-24042022-0108.arena_top.cropped_short.avi /adata/projects/autopi_mec/results/training_videos_arena_top/
cp /adata/projects/autopi_mec/jp1686/jp1686-29042022-0109/jp1686-29042022-0109.arena_top.cropped_short.avi /adata/projects/autopi_mec/results/training_vide

This can be copied offsite with `scp -r a230-pc59:/adata/projects/autopi_mec/results/training_videos_arena_top .`

I will use `review_labeled_video_and_extract` to extract frames with and without the mouse and difficult frames.

I have created a model called `arenaTopMouseLEDs_480_480-Allen-2022-09-29` and the instructions to train it will be stored in the model directory: `/adata/models/arenaTopMouseLEDs_480_480-Allen-2022-09-29`.

## Positrack2 

The online positrack tracking is very conservative and we often loose the mouse. 

I have trained a new model to track the mouse LEDs and the mouse head using the video saved by positrack2. 

We will use some video saved by positrack2. We get videos of open-field exploration, from the rest box and from the AutoPI task.

In [ ]:
load_parameters_from_files_project(sSessions)

100%|███████████████████████████████████████████| 92/92 [00:02<00:00, 32.26it/s]


In [16]:
sessionIndices = [1,13,25,37,49,73,85,90]
trialIndices = [0,1,4]

for i in sessionIndices:
    sSes = sSessions[i]
    for t in trialIndices:
        fn = "{}/{}{}".format(sSes.path,sSes.trial_names[t],".mp4")
        print(fn)

/adata/projects/autopi_mec/mn8578/mn8578-05122021-0108/mn8578-05122021_01.mp4
/adata/projects/autopi_mec/mn8578/mn8578-05122021-0108/mn8578-05122021_02.mp4
/adata/projects/autopi_mec/mn8578/mn8578-05122021-0108/mn8578-05122021_05.mp4
/adata/projects/autopi_mec/mn8578/mn8578-28122021-0107/mn8578-28122021_01.mp4
/adata/projects/autopi_mec/mn8578/mn8578-28122021-0107/mn8578-28122021_02.mp4
/adata/projects/autopi_mec/mn8578/mn8578-28122021-0107/mn8578-28122021_05.mp4
/adata/projects/autopi_mec/mn8599/mn8599-02022022-0107/mn8599-02022022_01.mp4
/adata/projects/autopi_mec/mn8599/mn8599-02022022-0107/mn8599-02022022_02.mp4
/adata/projects/autopi_mec/mn8599/mn8599-02022022-0107/mn8599-02022022_05.mp4
/adata/projects/autopi_mec/jp1084/jp1084-13042022-0108/jp1084-13042022_01.mp4
/adata/projects/autopi_mec/jp1084/jp1084-13042022-0108/jp1084-13042022_02.mp4
/adata/projects/autopi_mec/jp1084/jp1084-13042022-0108/jp1084-13042022_05.mp4
/adata/projects/autopi_mec/jp1685/jp1685-28042022-0108/jp1685-28

Copy these videos in a single directory for ease of copying across computers.

In [17]:
for i in sessionIndices:
    sSes = sSessions[i]
    for t in trialIndices:
        originFile = "{}/{}{}".format(sSes.path,sSes.trial_names[t],".mp4")
        destinationFolder = "/adata/projects/autopi_mec/results/training_videos_positrack2/"
        myCMD = "cp {} {}".format(originFile,destinationFolder)
        print(myCMD)
        os.system(myCMD)

cp /adata/projects/autopi_mec/mn8578/mn8578-05122021-0108/mn8578-05122021_01.mp4 /adata/projects/autopi_mec/results/training_videos_positrack2/
cp /adata/projects/autopi_mec/mn8578/mn8578-05122021-0108/mn8578-05122021_02.mp4 /adata/projects/autopi_mec/results/training_videos_positrack2/
cp /adata/projects/autopi_mec/mn8578/mn8578-05122021-0108/mn8578-05122021_05.mp4 /adata/projects/autopi_mec/results/training_videos_positrack2/
cp /adata/projects/autopi_mec/mn8578/mn8578-28122021-0107/mn8578-28122021_01.mp4 /adata/projects/autopi_mec/results/training_videos_positrack2/
cp /adata/projects/autopi_mec/mn8578/mn8578-28122021-0107/mn8578-28122021_02.mp4 /adata/projects/autopi_mec/results/training_videos_positrack2/
cp /adata/projects/autopi_mec/mn8578/mn8578-28122021-0107/mn8578-28122021_05.mp4 /adata/projects/autopi_mec/results/training_videos_positrack2/
cp /adata/projects/autopi_mec/mn8599/mn8599-02022022-0107/mn8599-02022022_01.mp4 /adata/projects/autopi_mec/results/training_videos_posi

This can be copied offsite with `scp -r a230-pc59:/adata/projects/autopi_mec/results/training_videos_positrack2 .`